## 4.1 FewShotPromptTemplate

In [ ]:
# api key 인증
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# t = PromptTemplate.from_template('What is the capital of {country}')

# t.format(country='France')

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example_template = """
#     Human: {question}
#     AI:{answer}
# """

# example_template = PromptTemplate.from_template(example_template)
example_prompt = PromptTemplate.from_template('Human : {question}\nAI:{answer}')

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human : What do you know about {country}?",
    input_variables=['country']
)

prompt.format(country='Germany')

chain = prompt | chat

chain.invoke({
    'country':'Korea'
})

AI:
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')

- 예제들을 가져오고 예제들을 prompt를 사용해서 형식화 해주고 그런 다음 이를 fewShotPromptTemplate에 전달
- fewShotPromptTemplate은 각각 예제들을 가져와 example prompt를 이용하여 형식화 해주고 그 다음 마지막으로 내용 마지막에 질문을 넣어줌

## 4.2 FewShotChatMessagePromptTemplate

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# t = PromptTemplate.from_template('What is the capital of {country}')

# t.format(country='France')

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example_template = """
#     Human: {question}
#     AI:{answer}
# """

# example_template = PromptTemplate.from_template(example_template)
example_prompt = ChatPromptTemplate.from_messages([
    ('human', 'What do you know about {country}?'),
    ('ai', '{answer}')
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a geography expert'),
    example_prompt,
    ('human', 'What do you know about {country}?')
])

chain = final_prompt | chat

chain.invoke({
    'country':'Korea'
})


        I know this:
        There are two Koreas - North Korea and South Korea.
        Capital of South Korea: Seoul
        Capital of North Korea: Pyongyang
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korea - South Korean Won, North Korea - North Korean Won
        

AIMessageChunk(content='\n        I know this:\n        There are two Koreas - North Korea and South Korea.\n        Capital of South Korea: Seoul\n        Capital of North Korea: Pyongyang\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korea - South Korean Won, North Korea - North Korean Won\n        ')

## 4.3 LengthBasedExampleSelector

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# t = PromptTemplate.from_template('What is the capital of {country}')

# t.format(country='France')

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example_template = """
#     Human: {question}
#     AI:{answer}
# """

# example_template = PromptTemplate.from_template(example_template)
example_prompt = PromptTemplate.from_template('Human : {question}\nAI:{answer}')

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=10,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human : What do you know about {country}?",
    input_variables=['country']
)

prompt.format(country='Germany')


'Human : What do you know about Germany?'

- 예제 sample 만들기

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

# t = PromptTemplate.from_template('What is the capital of {country}')

# t.format(country='France')

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    
    def __init__(self,examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import sample
        
        return sample(self.examples,k=1)

# example_template = """
#     Human: {question}
#     AI:{answer}
# """

# example_template = PromptTemplate.from_template(example_template)
example_prompt = PromptTemplate.from_template('Human : {question}\nAI:{answer}')

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human : What do you know about {country}?",
    input_variables=['country']
)

prompt.format(country='Germany')


'Human : What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman : What do you know about Germany?'